## Import Module

In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font',family='Malgun Gothic')

import warnings
warnings.filterwarnings("ignore")

In [108]:
train = pd.read_csv('L.POINT_train.csv', encoding='UTF-8')
test = pd.read_csv('L.POINT_test.csv', encoding='UTF-8')

In [109]:
train['PD_BUY_AM']  = train['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
train['PD_BUY_CT']  = train['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_AM']  = test['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_CT']  = test['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )

## Feature 

In [110]:
features = []
features_te = []

**[총구매액, 구매건수, 평균구매액, 최대구매액]**

In [111]:
train['AMOUNT'] = train['PD_BUY_AM'] * train['PD_BUY_CT']
test['AMOUNT'] = test['PD_BUY_AM'] * test['PD_BUY_CT']

In [112]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features.append(f);display(f)


f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features_te.append(f_te) ; display(f_te)

,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,0,86500,2,43250,81000
1,1,1276000,16,79750,99000
2,6,148900,7,21271,44900
3,9,42200,4,10550,12600
4,12,510180,32,15943,79000
...,...,...,...,...,...
149995,263094,10000,1,10000,10000
149996,263095,511000,5,102200,155000
149997,263096,85500,3,28500,47000
149998,263102,59400,15,3960,9720


,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,2,560000,11,50909,62100
1,3,851200,8,106400,110400
2,4,125200,6,20867,37000
3,5,380000,12,31667,42000
4,7,457900,16,28619,40000
...,...,...,...,...,...
113099,263097,753220,13,57940,105000
113100,263098,601200,6,100200,100200
113101,263099,973000,112,8688,22000
113102,263100,22400,2,11200,12900


**[주말방문비율]**

In [113]:
train['date'] = pd.to_datetime(train['SESS_DT'], format= '%Y%m%d')
test['date'] = pd.to_datetime(test['SESS_DT'], format= '%Y%m%d')

In [114]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features.append(f); display(f)


f_te = test.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features_te.append(f_te);display(f_te)

,CLNT_ID,주말방문비율
0,0,1.000000
1,1,0.125000
2,6,0.857143
3,9,1.000000
4,12,0.250000
...,...,...
149995,263094,0.000000
149996,263095,0.000000
149997,263096,0.000000
149998,263102,0.000000


,CLNT_ID,주말방문비율
0,2,0.000000
1,3,1.000000
2,4,0.333333
3,5,0.000000
4,7,0.312500
...,...,...
113099,263097,0.307692
113100,263098,0.000000
113101,263099,0.000000
113102,263100,0.500000


**[계절방문비율]**

In [115]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features.append(f); f

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,0,0.000,1.00000,0.00000,0.0
1,1,0.375,0.62500,0.00000,0.0
2,6,1.000,0.00000,0.00000,0.0
3,9,1.000,0.00000,0.00000,0.0
4,12,0.000,0.28125,0.71875,0.0
...,...,...,...,...,...
149995,263094,1.000,0.00000,0.00000,0.0
149996,263095,0.000,1.00000,0.00000,0.0
149997,263096,0.000,1.00000,0.00000,0.0
149998,263102,1.000,0.00000,0.00000,0.0


In [116]:
f_te = test.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features_te.append(f_te); f_te

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,2,0.000000,0.909091,0.090909,0.0
1,3,0.000000,0.000000,1.000000,0.0
2,4,0.666667,0.333333,0.000000,0.0
3,5,0.000000,0.000000,1.000000,0.0
4,7,0.437500,0.562500,0.000000,0.0
...,...,...,...,...,...
113099,263097,0.692308,0.307692,0.000000,0.0
113100,263098,0.000000,0.000000,1.000000,0.0
113101,263099,0.000000,1.000000,0.000000,0.0
113102,263100,0.500000,0.500000,0.000000,0.0


### Numeric Variable

**[평균대비 구매비중]**

In [117]:
f = (train.groupby('CLNT_ID')['AMOUNT'].sum() / train.AMOUNT.mean()).reset_index()
f = f.rename(columns = {'AMOUNT' : '평균대비구매비중'})
features.append(f)


f_te = (test.groupby('CLNT_ID')['AMOUNT'].sum() / test.AMOUNT.mean()).reset_index()
f_te = f_te.rename(columns = {'AMOUNT' : '평균대비구매비중'})
features_te.append(f);f_te

,CLNT_ID,평균대비구매비중
0,2,12.501198
1,3,19.001821
2,4,2.794911
3,5,8.482956
4,7,10.221962
...,...,...
113099,263097,16.814558
113100,263098,13.420929
113101,263099,21.720831
113102,263100,0.500048


**[검색건수]**

In [118]:
f = (train.groupby('CLNT_ID')['SEARCH_CNT'].count()).reset_index()
f = f.rename(columns = {'SEARCH_CNT' : '검색건수'})
features.append(f)


f_te = (test.groupby('CLNT_ID')['SEARCH_CNT'].count()).reset_index()
f_te = f_te.rename(columns = {'SEARCH_CNT' : '검색건수'})
features_te.append(f_te);f_te

,CLNT_ID,검색건수
0,2,11
1,3,8
2,4,6
3,5,12
4,7,16
...,...,...
113099,263097,13
113100,263098,6
113101,263099,112
113102,263100,2


**[검색건수비율]**

In [119]:
f = (train.groupby('CLNT_ID')['SEARCH_CNT'].count() / train['SEARCH_CNT'].mean()).reset_index()
f = f.rename(columns = {'SEARCH_CNT' : '검색건수비율'})
features.append(f)


f_te = (test.groupby('CLNT_ID')['SEARCH_CNT'].count() / test['SEARCH_CNT'].mean()).reset_index()
f_te = f_te.rename(columns = {'SEARCH_CNT' : '검색건수비율'})
features_te.append(f_te);f_te

,CLNT_ID,검색건수비율
0,2,6.127387
1,3,4.456282
2,4,3.342211
3,5,6.684423
4,7,8.912563
...,...,...
113099,263097,7.241458
113100,263098,3.342211
113101,263099,62.387944
113102,263100,1.114070


**[여성키워드상품구매건수]**

In [120]:
train['여성'] = train['CLAC1_NM'].str.startswith(('여성의류' , '화장품/뷰티케어' , '유아동의류','출산/육아용품', 
                                  '식기/조리기구' , '세제/위생' ,'인테리어/조명', 
                                  '청소/세탁/욕실용품' ,'원예/애완' , '생활/주방가전')).astype(int)
f = train[train.여성 ==1].groupby('CLNT_ID')['AMOUNT'].count().reset_index()
f = f.rename(columns = {'AMOUNT' : '여성키워드구매건수'})
features.append(f) 


test['여성'] = test['CLAC1_NM'].str.startswith(('여성의류' , '화장품/뷰티케어' , '유아동의류','출산/육아용품', 
                                  '식기/조리기구' , '세제/위생' ,'인테리어/조명', 
                                  '청소/세탁/욕실용품' ,'원예/애완' , '생활/주방가전')).astype(int)
f_te = test[test.여성 ==1].groupby('CLNT_ID')['AMOUNT'].count().reset_index()
f_te = f_te.rename(columns = {'AMOUNT' : '여성키워드구매건수'})
features_te.append(f_te);f_te

,CLNT_ID,여성키워드구매건수
0,2,1
1,4,2
2,5,12
3,7,13
4,11,2
...,...,...
65629,263084,2
65630,263085,5
65631,263089,3
65632,263097,3


**[남성키워드구매건수]**

In [121]:
train['남성'] = train['CLAC1_NM'].str.startswith(('남성의류','시즌스포츠','아웃도어/레저',
                                               '구기/필드스포츠','모바일','컴퓨터',
                                               '영상/음향가전','자동차용품')).astype(int)
f = train[train.남성 ==1].groupby('CLNT_ID')['AMOUNT'].count().reset_index()
f = f.rename(columns = {'AMOUNT' : '남성키워드구매건수'})
features.append(f)

test['남성'] = test['CLAC1_NM'].str.startswith(('남성의류','시즌스포츠','아웃도어/레저',
                                               '구기/필드스포츠','모바일','컴퓨터',
                                               '영상/음향가전','자동차용품')).astype(int)
f_te = test[test.남성 ==1].groupby('CLNT_ID')['AMOUNT'].count().reset_index()
f_te = f_te.rename(columns = {'AMOUNT' : '남성키워드구매건수'})
features_te.append(f_te);f_te

,CLNT_ID,남성키워드구매건수
0,4,4
1,7,2
2,8,12
3,10,4
4,13,3
...,...,...
31880,263097,3
31881,263098,6
31882,263099,112
31883,263100,1


**[총페이지조회건수비율]**

In [122]:
f = (train.groupby('CLNT_ID')['TOT_PAG_VIEW_CT'].count() / train['TOT_PAG_VIEW_CT'].mean()).reset_index()
f = f.rename(columns = {'TOT_PAG_VIEW_CT' : '총페이지조회건수비율'})
features.append(f)


f_te = (test.groupby('CLNT_ID')['TOT_PAG_VIEW_CT'].count() / test['TOT_PAG_VIEW_CT'].mean()).reset_index()
f_te = f_te.rename(columns = {'TOT_PAG_VIEW_CT' : '총페이지조회건수비율'})
features_te.append(f_te);f_te

,CLNT_ID,총페이지조회건수비율
0,2,0.051191
1,3,0.037230
2,4,0.027922
3,5,0.055844
4,7,0.074459
...,...,...
113099,263097,0.060498
113100,263098,0.027922
113101,263099,0.521215
113102,263100,0.009307


**[평일방문비율]**

In [123]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('평일방문비율', lambda x: np.mean(x.dt.dayofweek<=4))]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['date'].agg([
    ('평일방문비율', lambda x: np.mean(x.dt.dayofweek<=4))]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,평일방문비율
0,2,1.000000
1,3,0.000000
2,4,0.666667
3,5,1.000000
4,7,0.687500
...,...,...
113099,263097,0.692308
113100,263098,1.000000
113101,263099,1.000000
113102,263100,0.500000


**[접속건수]**

In [124]:
f = (train.groupby('CLNT_ID')['date'].agg(lambda x : x.nunique())).reset_index()
f = f.rename(columns = {'date' : '접속건수'})
features.append(f)


f_te = (test.groupby('CLNT_ID')['date'].agg(lambda x : x.nunique())).reset_index()
f_te = f_te.rename(columns = {'date' : '접속건수'})
features_te.append(f_te);f_te

,CLNT_ID,접속건수
0,2,2
1,3,1
2,4,2
3,5,1
4,7,5
...,...,...
113099,263097,2
113100,263098,1
113101,263099,1
113102,263100,2


**[접속시구매금액]**

In [125]:
a = train.groupby('CLNT_ID')['date'].agg(lambda x: x.nunique())
f = (train.groupby('CLNT_ID')['AMOUNT'].sum() / a).reset_index().rename(columns={0 : "접속시구매액"})
features.append(f)

a_te= train.groupby('CLNT_ID')['date'].agg(lambda x: x.nunique())
f_te = (train.groupby('CLNT_ID')['AMOUNT'].sum() / a_te).reset_index().rename(columns={0 : "접속시구매액"})
features_te.append(f_te);f_te

,CLNT_ID,접속시구매액
0,0,86500.000000
1,1,319000.000000
2,6,74450.000000
3,9,42200.000000
4,12,102036.000000
...,...,...
149995,263094,10000.000000
149996,263095,255500.000000
149997,263096,85500.000000
149998,263102,59400.000000


**[접속시구매건수]**

In [126]:
a = train.groupby('CLNT_ID')['date'].agg(lambda x: x.nunique())
f = (train.groupby('CLNT_ID')['AMOUNT'].size() / a).reset_index().rename(columns={0 : "접속시구매건수"})
features.append(f)


a_te = test.groupby('CLNT_ID')['date'].agg(lambda x: x.nunique())
f_te = (test.groupby('CLNT_ID')['AMOUNT'].size() / a_te).reset_index().rename(columns={0 : "접속시구매건수"})
features_te.append(f_te);f_te

,CLNT_ID,접속시구매건수
0,2,5.5
1,3,8.0
2,4,3.0
3,5,12.0
4,7,3.2
...,...,...
113099,263097,6.5
113100,263098,6.0
113101,263099,112.0
113102,263100,1.0


**[접속지역수]**

In [127]:
f = train.groupby('CLNT_ID')['CITY_NM'].agg([
    ('접속지역수',  lambda x : x.nunique())]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['CITY_NM'].agg([
    ('접속지역수',  lambda x : x.nunique())]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,접속지역수
0,2,1
1,3,1
2,4,1
3,5,1
4,7,2
...,...,...
113099,263097,1
113100,263098,1
113101,263099,1
113102,263100,1


**[총세션시간]**

In [128]:
f = train.groupby('CLNT_ID')['TOT_SESS_HR_V'].agg([
    ('총세션시간' , lambda x : x.map(lambda x: int(str(x).replace(',','')) ).sum())]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['TOT_SESS_HR_V'].agg([
    ('총세션시간' , lambda x : x.map(lambda x: int(str(x).replace(',','')) ).sum())]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,총세션시간
0,2,48241
1,3,8408
2,4,4198
3,5,38568
4,7,71503
...,...,...
113099,263097,43079
113100,263098,7050
113101,263099,435680
113102,263100,2949


**[구매브랜드수]**

In [129]:
f = train.groupby('CLNT_ID')['PD_BRA_NM'].agg([
    ('구매브랜드수' , lambda x : x.nunique())]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['PD_BRA_NM'].agg([
    ('구매브랜드수' , lambda x : x.nunique())]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,구매브랜드수
0,2,3
1,3,2
2,4,2
3,5,2
4,7,7
...,...,...
113099,263097,4
113100,263098,1
113101,263099,1
113102,263100,2


**[검색키워드수]**

In [130]:
f = train.groupby('CLNT_ID')['KWD_NM'].agg([
    ('검색키워드수' , lambda x : x.nunique())]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['KWD_NM'].agg([
    ('검색키워드수' , lambda x : x.nunique())]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,검색키워드수
0,2,6
1,3,4
2,4,4
3,5,4
4,7,10
...,...,...
113099,263097,7
113100,263098,6
113101,263099,7
113102,263100,2


**[사용기기수]**

In [131]:
f = train.groupby('CLNT_ID')['DVC_CTG_NM'].agg([
    ('사용기기수' , lambda x : x.nunique())]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['DVC_CTG_NM'].agg([
    ('사용기기수' , lambda x : x.nunique())]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,사용기기수
0,2,1
1,3,1
2,4,1
3,5,1
4,7,1
...,...,...
113099,263097,1
113100,263098,1
113101,263099,1
113102,263100,1


**[사이트접속기간]**

In [132]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('사이트접속기간' , lambda x : int((x.max() - x.min()).days) )]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['date'].agg([
    ('사이트접속기간' , lambda x : int((x.max() - x.min()).days) )]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,사이트접속기간
0,2,81
1,3,0
2,4,102
3,5,0
4,7,82
...,...,...
113099,263097,51
113100,263098,0
113101,263099,0
113102,263100,25


**[총페이지조회건수, 평균조회건수]**

In [133]:
f = train.groupby('CLNT_ID')['TOT_PAG_VIEW_CT'].agg([
     ('총페이지조회건수', lambda x : x.sum()),
     ('평균조회건수', lambda x : np.mean(x))]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['TOT_PAG_VIEW_CT'].agg([
     ('총페이지조회건수', lambda x : x.sum()),
     ('평균조회건수', lambda x : np.mean(x))]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,총페이지조회건수,평균조회건수
0,2,2906.0,264.181818
1,3,744.0,93.000000
2,4,340.0,56.666667
3,5,612.0,51.000000
4,7,4535.0,283.437500
...,...,...,...
113099,263097,6022.0,463.230769
113100,263098,816.0,136.000000
113101,263099,30688.0,274.000000
113102,263100,208.0,104.000000


**[평균세션시간]**

In [134]:
train['TOT_SESS_HR_V'] = train['TOT_SESS_HR_V'].map(lambda x: int(str(x).replace(',','')) )
test['TOT_SESS_HR_V'] = test['TOT_SESS_HR_V'].map(lambda x: int(str(x).replace(',','')) )


f = train.groupby('CLNT_ID')['TOT_SESS_HR_V'].agg([
        ('평균세션시간', lambda x : np.mean(x))]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['TOT_SESS_HR_V'].agg([
        ('평균세션시간', lambda x : np.mean(x))]).reset_index()
features_te.append(f_te);f_te  

,CLNT_ID,평균세션시간
0,2,4385.545455
1,3,1051.000000
2,4,699.666667
3,5,3214.000000
4,7,4468.937500
...,...,...
113099,263097,3313.769231
113100,263098,1175.000000
113101,263099,3890.000000
113102,263100,1474.500000


**[총검색건수 , 평균검색건수]**

In [135]:
f = train.groupby('CLNT_ID')['SEARCH_CNT'].agg([
    ('총검색건수', lambda x : x.sum()),
    ('평균검색건수', lambda x : np.mean(x))]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['SEARCH_CNT'].agg([
    ('총검색건수', lambda x : x.sum()),
    ('평균검색건수', lambda x : np.mean(x))]).reset_index()

features_te.append(f_te); f_te

,CLNT_ID,총검색건수,평균검색건수
0,2,23,2.090909
1,3,12,1.500000
2,4,11,1.833333
3,5,15,1.250000
4,7,16,1.000000
...,...,...,...
113099,263097,44,3.384615
113100,263098,7,1.166667
113101,263099,128,1.142857
113102,263100,2,1.000000


**[계절별 총 구매금액]**

In [136]:
def season(x):
    if 3 <= x <= 5:
        return '봄'
    elif 6 <= x <= 8:
        return '여름'
    elif 9 <= x <= 11:
        return '가을'
    else:
        return '겨울'
    
train['SEASON'] = train['date'].dt.month.apply(season)
test['SEASON'] = test['date'].dt.month.apply(season)


f = pd.pivot_table(train, index = 'CLNT_ID', columns = 'SEASON', 
                   values = 'AMOUNT', aggfunc = sum).fillna(0).reset_index().rename(columns = {'가을' : '가을-총구매액', '봄' : '봄-총구매액', '여름' : '여름-총구매액', '겨울': '겨울-총구매액'})
features.append(f)


f_te = pd.pivot_table(test, index = 'CLNT_ID', columns = 'SEASON', 
                      values = 'AMOUNT', aggfunc = sum).fillna(0).reset_index().rename(columns = {'가을' : '가을-총구매액', '봄' : '봄-총구매액', '여름' : '여름-총구매액', '겨울' : '겨울-총구매액'})
features_te.append(f_te); f_te

SEASON,CLNT_ID,가을-총구매액,봄-총구매액,여름-총구매액
0,2,20000.0,0.0,540000.0
1,3,851200.0,0.0,0.0
2,4,0.0,51200.0,74000.0
3,5,380000.0,0.0,0.0
4,7,0.0,180400.0,277500.0
...,...,...,...,...
113099,263097,0.0,649620.0,103600.0
113100,263098,601200.0,0.0,0.0
113101,263099,0.0,0.0,973000.0
113102,263100,0.0,12900.0,9500.0


**[계절별 평균구매액]**

In [137]:
f = pd.pivot_table(train, index = 'CLNT_ID', columns = 'SEASON', 
                   values = 'AMOUNT', aggfunc = 'mean').fillna(0).reset_index().rename(columns = {'가을' : '가을-평균구매액', '봄' : '봄-평균구매액', '여름' : '여름-평균구매액'})
features.append(f)


f_te = pd.pivot_table(test, index = 'CLNT_ID', columns = 'SEASON', 
                   values = 'AMOUNT', aggfunc = 'mean').fillna(0).reset_index().rename(columns = {'가을' : '가을-평균구매액', '봄' : '봄-평균구매액', '여름' : '여름-평균구매액'})
features_te.append(f_te); f_te

SEASON,CLNT_ID,가을-평균구매액,봄-평균구매액,여름-평균구매액
0,2,20000.000000,0.000000,54000.000000
1,3,106400.000000,0.000000,0.000000
2,4,0.000000,12800.000000,37000.000000
3,5,31666.666667,0.000000,0.000000
4,7,0.000000,25771.428571,30833.333333
...,...,...,...,...
113099,263097,0.000000,72180.000000,25900.000000
113100,263098,100200.000000,0.000000,0.000000
113101,263099,0.000000,0.000000,8687.500000
113102,263100,0.000000,12900.000000,9500.000000


**[남성,여성관련상품 구매율]**

In [138]:
f = train[['CLNT_ID']]
f['남성관련상품구매'] = train['CLAC3_NM'].str.contains('남성')
f['여성관련상품구매'] = train['CLAC3_NM'].str.contains('여성')


f1 = f.groupby('CLNT_ID')['여성관련상품구매'].agg([('여성관련상품구매비율',np.mean)]).reset_index()
f1['남성관련상품구매비율'] = f.groupby('CLNT_ID')['남성관련상품구매'].mean()
features.append(f1) ; display(f1)


f_te = test[['CLNT_ID']]
f_te['남성관련상품구매'] = test['CLAC3_NM'].str.contains('남성')
f_te['여성관련상품구매'] = test['CLAC3_NM'].str.contains('여성')


t1 = f_te.groupby('CLNT_ID')['여성관련상품구매'].agg([('여성관련상품구매비율',np.mean)]).reset_index()
t1['남성관련상품구매비율'] = f_te.groupby('CLNT_ID')['남성관련상품구매'].mean()
features_te.append(t1) ; display(t1)

,CLNT_ID,여성관련상품구매비율,남성관련상품구매비율
0,0,0.000000,0.000000
1,1,0.187500,0.062500
2,6,0.714286,NaN
3,9,0.000000,NaN
4,12,0.093750,NaN
...,...,...,...
149995,263094,1.000000,0.666667
149996,263095,0.000000,0.000000
149997,263096,0.333333,0.171429
149998,263102,0.000000,0.285714


,CLNT_ID,여성관련상품구매비율,남성관련상품구매비율
0,2,0.000000,NaN
1,3,1.000000,NaN
2,4,0.000000,0.000000
3,5,0.000000,0.000000
4,7,0.812500,0.666667
...,...,...,...
113099,263097,0.307692,NaN
113100,263098,0.000000,0.000000
113101,263099,0.000000,NaN
113102,263100,0.500000,NaN


**[구매한상품 대,중,소분류 개수]**

In [139]:
f = train.groupby('CLNT_ID')['CLAC1_NM'].agg([('구매상품종류(대분류)', 'count')]).reset_index()
f['구매상품종류(중분류)'] = train.groupby('CLNT_ID')['CLAC2_NM'].count()
f['구매상품종류(소분류)'] = train.groupby('CLNT_ID')['CLAC3_NM'].count()
features.append(f)


f_te = test.groupby('CLNT_ID')['CLAC1_NM'].agg([('구매상품종류(대분류)', 'count')]).reset_index()
f_te['구매상품종류(중분류)'] = test.groupby('CLNT_ID')['CLAC2_NM'].count()
f_te['구매상품종류(소분류)'] = test.groupby('CLNT_ID')['CLAC3_NM'].count()
features_te.append(f_te); f_te

,CLNT_ID,구매상품종류(대분류),구매상품종류(중분류),구매상품종류(소분류)
0,2,11,NaN,NaN
1,3,8,NaN,NaN
2,4,6,11.0,11.0
3,5,12,8.0,8.0
4,7,16,6.0,6.0
...,...,...,...,...
113099,263097,13,NaN,NaN
113100,263098,6,3.0,3.0
113101,263099,112,NaN,NaN
113102,263100,2,NaN,NaN


**[구매상품 중 최고가상품 금액]**

In [140]:
f = train.groupby('CLNT_ID')['PD_BUY_AM'].agg([('최고가상품금액', 'max')]).reset_index()
features.append(f); display(f)


f_te = test.groupby('CLNT_ID')['PD_BUY_AM'].agg([('최고가상품금액', 'max')]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,최고가상품금액
0,0,81000
1,1,99000
2,6,44900
3,9,12600
4,12,79000
...,...,...
149995,263094,10000
149996,263095,155000
149997,263096,47000
149998,263102,1080


,CLNT_ID,최고가상품금액
0,2,62100
1,3,110400
2,4,37000
3,5,42000
4,7,40000
...,...,...
113099,263097,105000
113100,263098,100200
113101,263099,22000
113102,263100,12900


**[고가상품구매율]**

In [141]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('구매건수', np.size)]).reset_index()
f1 = train.loc[train['PD_BUY_AM'] > train['PD_BUY_AM'].quantile(0.95)].groupby('CLNT_ID')['PD_BUY_AM'].agg([('고가상품구매건수',np.size)]).reset_index()
f = pd.merge(f,f1, on = 'CLNT_ID', how='left').fillna(0)
f['고가상품구매율'] = f['고가상품구매건수'] / f['구매건수'] 
features.append(f[['CLNT_ID','고가상품구매율']]); f[['CLNT_ID','고가상품구매율']]


f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('구매건수', np.size)]).reset_index()
f_te1 = test.loc[train['PD_BUY_AM'] > test['PD_BUY_AM'].quantile(0.95)].groupby('CLNT_ID')['PD_BUY_AM'].agg([('고가상품구매건수',np.size)]).reset_index()
f_te = pd.merge(f_te,f_te1, on = 'CLNT_ID', how='left').fillna(0)
f_te['고가상품구매율'] = f_te['고가상품구매건수'] / f_te['구매건수'] 
features_te.append(f_te[['CLNT_ID','고가상품구매율']]); f_te[['CLNT_ID','고가상품구매율']]

,CLNT_ID,고가상품구매율
0,2,0.000000
1,3,0.000000
2,4,0.000000
3,5,0.000000
4,7,0.000000
...,...,...
113099,263097,0.000000
113100,263098,0.000000
113101,263099,0.044643
113102,263100,0.000000


**[구매주기]**

In [142]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('구매주기', lambda x: int(((x.max() - x.min()).days) / x.nunique()))]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['date'].agg([
    ('구매주기', lambda x: int(((x.max() - x.min()).days) / x.nunique()))]).reset_index()
features_te.append(f_te); f_te

,CLNT_ID,구매주기
0,2,40
1,3,0
2,4,51
3,5,0
4,7,16
...,...,...
113099,263097,25
113100,263098,0
113101,263099,0
113102,263100,12


**[저가상품구매율]**

In [143]:
a = train.groupby('CLNT_ID')['AMOUNT'].agg([('건수' , 'count')]).reset_index()
b = train.loc[train['PD_BUY_AM'] <= train['PD_BUY_AM'].quantile(0.2)].groupby('CLNT_ID')['PD_BUY_AM'].agg([
    ('저가' , 'count')]).reset_index()
f = pd.merge(a,b, on='CLNT_ID',how='left').fillna(0)
f['저가상품구매율'] = f.저가 / f.건수;f
features.append(f[['CLNT_ID','저가상품구매율']])


a_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('건수' , 'count')]).reset_index()
b_te = test.loc[train['PD_BUY_AM'] <= test['PD_BUY_AM'].quantile(0.2)].groupby('CLNT_ID')['PD_BUY_AM'].agg([
    ('저가' , 'count')]).reset_index()
f_te = pd.merge(a,b, on='CLNT_ID',how='left').fillna(0)
f_te['저가상품구매율'] = f_te.저가 / f_te.건수;f
features_te.append(f_te[['CLNT_ID','저가상품구매율']]); f_te[['CLNT_ID','저가상품구매율']]

,CLNT_ID,저가상품구매율
0,0,0.500000
1,1,0.000000
2,6,0.285714
3,9,0.500000
4,12,0.718750
...,...,...
149995,263094,1.000000
149996,263095,0.000000
149997,263096,0.000000
149998,263102,1.000000


**[베스트셀러 구매율 CLAC_1,2,3]**

In [144]:
train.CLAC1_NM.value_counts()[:5] , test.CLAC1_NM.value_counts()[:5]

(여성의류        252586
 화장품/뷰티케어    240390
 스포츠패션       222749
 남성의류        204666
 유아동의류       183804
 Name: CLAC1_NM, dtype: int64,
 여성의류        194290
 화장품/뷰티케어    179782
 스포츠패션       165477
 남성의류        153692
 유아동의류       138016
 Name: CLAC1_NM, dtype: int64)

In [145]:
f = train.groupby('CLNT_ID')['CLAC1_NM'].agg([
    ('대분류베스트구매율' , lambda x : np.mean(x.isin([
        '여성의류' , '화장품/뷰티케어' , '스포츠패션' , '남성의류' , '유아동의류'])))]).reset_index()
features.append(f);f


f_te = test.groupby('CLNT_ID')['CLAC1_NM'].agg([
    ('대분류베스트구매율' , lambda x : np.mean(x.isin([
        '여성의류' , '화장품/뷰티케어' , '스포츠패션' , '남성의류' , '유아동의류'])))]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,대분류베스트구매율
0,2,0.909091
1,3,0.000000
2,4,1.000000
3,5,1.000000
4,7,1.000000
...,...,...
113099,263097,0.538462
113100,263098,1.000000
113101,263099,1.000000
113102,263100,1.000000


In [146]:
train.CLAC2_NM.value_counts()[:5] , test.CLAC2_NM.value_counts()[:5]

(남성의류상의    147156
 여성의류상의    119830
 메이크업      108105
 스킨케어       93240
 수영/물놀이     68602
 Name: CLAC2_NM, dtype: int64,
 남성의류상의    110353
 여성의류상의     92490
 메이크업       82150
 스킨케어       68344
 수영/물놀이     51349
 Name: CLAC2_NM, dtype: int64)

In [147]:
f = train.groupby('CLNT_ID')['CLAC2_NM'].agg([
    ('중분류베스트구매율' , lambda x : np.mean(x.isin([
        '남성의류상의' , '여성의류상의' , '메이크업' , '스킨케어' , '수영/물놀이'])))]).reset_index()
features.append(f);f


f_te = test.groupby('CLNT_ID')['CLAC2_NM'].agg([
    ('중분류베스트구매율' , lambda x : np.mean(x.isin([
        '남성의류상의' , '여성의류상의' , '메이크업' , '스킨케어' , '수영/물놀이'])))]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,중분류베스트구매율
0,2,0.000000
1,3,0.000000
2,4,1.000000
3,5,0.666667
4,7,0.500000
...,...,...
113099,263097,0.230769
113100,263098,1.000000
113101,263099,0.812500
113102,263100,0.500000


In [148]:
train.CLAC3_NM.value_counts()[:5] , test.CLAC3_NM.value_counts()[:5]

(남성티셔츠      121339
 여성원피스       58653
 여성티셔츠/탑     52560
 여성남방셔츠      45828
 남성캐주얼바지     41673
 Name: CLAC3_NM, dtype: int64,
 남성티셔츠      90763
 여성원피스      44700
 여성티셔츠/탑    40706
 여성남방셔츠     34561
 남성캐주얼바지    31307
 Name: CLAC3_NM, dtype: int64)

In [149]:
f = train.groupby('CLNT_ID')['CLAC3_NM'].agg([
    ('소분류베스트구매율' , lambda x : np.mean(x.isin([
        '남성티셔츠' , '여성원피스' , '여성티셔츠/탑' , '여성남방셔츠' , '남성캐주얼바지'])))]).reset_index()
features.append(f);f


f_te = test.groupby('CLNT_ID')['CLAC3_NM'].agg([
    ('소분류베스트구매율' , lambda x : np.mean(x.isin([
        '남성티셔츠' , '여성원피스' , '여성티셔츠/탑' , '여성남방셔츠' , '남성캐주얼바지'])))]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,소분류베스트구매율
0,2,0.000000
1,3,0.000000
2,4,0.666667
3,5,0.000000
4,7,0.500000
...,...,...
113099,263097,0.000000
113100,263098,0.000000
113101,263099,1.000000
113102,263100,0.500000


**[선호가격]**

In [150]:
train.AMOUNT.describe()

count    1.948686e+06
mean     4.561267e+04
std      9.567259e+04
min      1.000000e+02
25%      1.380000e+04
50%      2.800000e+04
75%      4.900000e+04
max      1.223400e+07
Name: AMOUNT, dtype: float64

In [151]:
def price(x):
    if x <= 13800:
        return 1
    elif x <= 28000:
        return 2
    elif x <= 49000:
        return 3
    else:
        return 4

In [152]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([
    ('선호가격' , lambda x : price(x.mean()))]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([
    ('선호가격' , lambda x : price(x.mean()))]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,선호가격
0,2,4
1,3,4
2,4,2
3,5,3
4,7,3
...,...,...
113099,263097,4
113100,263098,4
113101,263099,1
113102,263100,1


**[주요고객지수]**

In [153]:
train.AMOUNT.quantile(0.9)

89000.0

In [154]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([
    ('주요고객지수' , lambda x : x.sum() / 89000)]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([
    ('주요고객지수' , lambda x : x.sum()/ 89000)]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,주요고객지수
0,2,6.292135
1,3,9.564045
2,4,1.406742
3,5,4.269663
4,7,5.144944
...,...,...
113099,263097,8.463146
113100,263098,6.755056
113101,263099,10.932584
113102,263100,0.251685


**[평균구매주기]**

In [155]:
f = ((train.groupby('CLNT_ID')['date'].max() - train.groupby('CLNT_ID')['date'].min()).apply(lambda x : x.days) / train.groupby('CLNT_ID')['date'].nunique()).reset_index()
f = f.rename(columns = {'date' : '평균구매주기'})
features.append(f)
display(f)


f_te = ((test.groupby('CLNT_ID')['date'].max() - test.groupby('CLNT_ID')['date'].min()).apply(lambda x : x.days) / test.groupby('CLNT_ID')['date'].nunique()).reset_index()
f_te = f_te.rename(columns = {'date' : '평균구매주기'})
features_te.append(f_te)
display(f_te)

,CLNT_ID,평균구매주기
0,0,0.000000
1,1,7.250000
2,6,8.500000
3,9,0.000000
4,12,12.600000
...,...,...
149995,263094,0.000000
149996,263095,41.500000
149997,263096,0.000000
149998,263102,0.000000


,CLNT_ID,평균구매주기
0,2,40.5
1,3,0.0
2,4,51.0
3,5,0.0
4,7,16.4
...,...,...
113099,263097,25.5
113100,263098,0.0
113101,263099,0.0
113102,263100,12.5


**[아동관련용품구매액비율]**

In [156]:
baby = ['유아동의류', '출산/육아용품', '완구']

baby_sum = train.query("CLAC1_NM in @baby").groupby('CLNT_ID')['AMOUNT'].agg([('아동관련용품구매액비율', 'sum')]).reset_index()
f = pd.merge(pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()}), baby_sum, how = 'outer').fillna(0)
f['아동관련용품구매액비율'] = f['아동관련용품구매액비율'] / (train.groupby('CLNT_ID')['AMOUNT'].sum())
f = f.fillna(0)


features.append(f)
display(f)


baby_sum = test.query("CLAC1_NM in @baby").groupby('CLNT_ID')['AMOUNT'].agg([('아동관련용품구매액비율', 'sum')]).reset_index()
f_te = pd.merge(pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()}), baby_sum, how = 'outer').fillna(0)
f_te['아동관련용품구매액비율'] = f_te['아동관련용품구매액비율'] / (test.groupby('CLNT_ID')['AMOUNT'].sum())
f_te = f_te.fillna(0)


features_te.append(f_te)
display(f_te)

,CLNT_ID,아동관련용품구매액비율
0,0,0.000000
1,1,0.000000
2,6,0.000000
3,9,0.000000
4,12,0.000000
...,...,...
149995,263094,0.000000
149996,263095,49.444444
149997,263096,0.000000
149998,263102,0.000000


,CLNT_ID,아동관련용품구매액비율
0,2,0.000000
1,3,0.000000
2,10,0.000000
3,15,0.142505
4,29,0.000000
...,...,...
113099,263089,0.000000
113100,263097,0.000000
113101,263098,0.000000
113102,263099,0.000000


**[스포츠용품구매액비율]**

In [157]:
sport = ['시즌스포츠', '스포츠패션', '구기/필드스포츠', '아웃도어/레저', '헬스/피트니스']


sport_sum = train.query("CLAC1_NM in @sport").groupby('CLNT_ID')['AMOUNT'].agg([('스포츠용품구매액비율', 'sum')]).reset_index()
f = pd.merge(pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()}), sport_sum, how = 'outer').fillna(0)
f['스포츠용품구매액비율'] = f['스포츠용품구매액비율'] / (train.groupby('CLNT_ID')['AMOUNT'].sum())
f = f.fillna(0)


features.append(f)
display(f)


sport_sum = test.query("CLAC1_NM in @sport").groupby('CLNT_ID')['AMOUNT'].agg([('스포츠용품구매액비율', 'sum')]).reset_index()
f_te = pd.merge(pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()}), sport_sum, how = 'outer').fillna(0)
f_te['스포츠용품구매액비율'] = f_te['스포츠용품구매액비율'] / (test.groupby('CLNT_ID')['AMOUNT'].sum())
f_te = f_te.fillna(0)


features_te.append(f_te)
display(f_te)

,CLNT_ID,스포츠용품구매액비율
0,0,0.000000
1,1,0.278997
2,6,0.000000
3,9,0.000000
4,12,0.000000
...,...,...
149995,263094,0.000000
149996,263095,0.000000
149997,263096,0.014415
149998,263102,0.000000


,CLNT_ID,스포츠용품구매액비율
0,2,0.000000
1,3,0.000000
2,10,0.000000
3,15,0.049342
4,29,0.000000
...,...,...
113099,263089,0.000000
113100,263097,5.040662
113101,263098,0.000000
113102,263099,0.000000


**[휴면일수]**

In [158]:
import datetime as dt

time = dt.datetime(2018,9,30)

f = train.groupby('CLNT_ID')['date'].agg([('휴면일수', lambda x : (time - x.astype('datetime64').max()).days)]).reset_index(); f
features.append(f) ; f


f_te = test.groupby('CLNT_ID')['date'].agg([('휴면일수', lambda x : (time - x.astype('datetime64').max()).days)]).reset_index(); f
features_te.append(f_te) ; f_te

,CLNT_ID,휴면일수
0,2,27
1,3,21
2,4,50
3,5,19
4,7,89
...,...,...
113099,263097,98
113100,263098,18
113101,263099,90
113102,263100,99


**[상/하반기 구매비율 및 추세]**

In [159]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('상반기구매비율', lambda x: np.mean(x.dt.month.isin([1, 2, 3, 4, 5, 6]))),
    ('하반기구매비율', lambda x: np.mean(x.dt.month.isin([7, 8, 9, 10, 11, 12])))
]).reset_index()
f['구매추세'] = f['하반기구매비율'] - f['상반기구매비율']


features.append(f) ; display(f)


f_te = test.groupby('CLNT_ID')['date'].agg([
    ('상반기구매비율', lambda x: np.mean(x.dt.month.isin([1, 2, 3, 4, 5, 6]))),
    ('하반기구매비율', lambda x: np.mean(x.dt.month.isin([7, 8, 9, 10, 11, 12])))
]).reset_index()
f_te['구매추세'] = f_te['하반기구매비율'] - f_te['상반기구매비율']


features_te.append(f_te) ; display(f_te)

,CLNT_ID,상반기구매비율,하반기구매비율,구매추세
0,0,1.0,0.0,-1.0
1,1,1.0,0.0,-1.0
2,6,1.0,0.0,-1.0
3,9,1.0,0.0,-1.0
4,12,0.0,1.0,1.0
...,...,...,...,...
149995,263094,1.0,0.0,-1.0
149996,263095,0.8,0.2,-0.6
149997,263096,0.0,1.0,1.0
149998,263102,1.0,0.0,-1.0


,CLNT_ID,상반기구매비율,하반기구매비율,구매추세
0,2,0.909091,0.090909,-0.818182
1,3,0.000000,1.000000,1.000000
2,4,0.666667,0.333333,-0.333333
3,5,0.000000,1.000000,1.000000
4,7,0.500000,0.500000,0.000000
...,...,...,...,...
113099,263097,1.000000,0.000000,-1.000000
113100,263098,0.000000,1.000000,1.000000
113101,263099,0.000000,1.000000,1.000000
113102,263100,1.000000,0.000000,-1.000000


**[건강식품 구매비율(건수)]**

In [160]:
f = train.groupby('CLNT_ID')['CLAC1_NM'].agg([('건강식품구매건수', lambda x: list(x).count('건강식품')),
                                             ('구매건수',np.size)]).reset_index()

f['건강식품구매비율'] = f['건강식품구매건수']/f['구매건수']
features.append(f[['CLNT_ID','건강식품구매비율']]) ; display(f[['CLNT_ID','건강식품구매비율']])


f_te = test.groupby('CLNT_ID')['CLAC1_NM'].agg([('건강식품구매건수', lambda x: list(x).count('건강식품')),
                                             ('구매건수',np.size)]).reset_index()

f_te['건강식품구매비율'] = f_te['건강식품구매건수']/f_te['구매건수']
features_te.append(f_te[['CLNT_ID','건강식품구매비율']]); display(f_te[['CLNT_ID','건강식품구매비율']])

,CLNT_ID,건강식품구매비율
0,0,0.000
1,1,0.125
2,6,0.000
3,9,0.000
4,12,0.000
...,...,...
149995,263094,0.000
149996,263095,0.000
149997,263096,0.000
149998,263102,0.000


,CLNT_ID,건강식품구매비율
0,2,0.000000
1,3,0.000000
2,4,0.000000
3,5,0.000000
4,7,0.000000
...,...,...
113099,263097,0.230769
113100,263098,0.000000
113101,263099,0.000000
113102,263100,0.000000


**[건강식품 구매비율(금액)]**

In [161]:
f = train.groupby("CLNT_ID")['AMOUNT'].agg([('총구매액', np.sum)])
f1 = train.loc[train['CLAC1_NM']=='건강식품'].groupby('CLNT_ID')['AMOUNT'].agg([('건강식품구매금액', np.sum)]).reset_index()
f = pd.merge(f,f1, on = 'CLNT_ID', how='left').fillna(0)
f['건강식품구매비율(금액)'] = f['건강식품구매금액']/f['총구매액']
features.append(f[['CLNT_ID','건강식품구매비율(금액)']]) ; display(f[['CLNT_ID','건강식품구매비율(금액)']])


f_te = test.groupby("CLNT_ID")['AMOUNT'].agg([('총구매액', np.sum)])
f1_te = test.loc[test['CLAC1_NM']=='건강식품'].groupby('CLNT_ID')['AMOUNT'].agg([('건강식품구매금액', np.sum)]).reset_index()
f_te = pd.merge(f_te,f1_te, on = 'CLNT_ID', how='left').fillna(0)
f_te['건강식품구매비율(금액)'] = f_te['건강식품구매금액']/f_te['총구매액']
f_te.drop
features_te.append(f_te[['CLNT_ID','건강식품구매비율(금액)']]) ; display(f_te[['CLNT_ID','건강식품구매비율(금액)']])

,CLNT_ID,건강식품구매비율(금액)
0,0,0.00000
1,1,0.15047
2,6,0.00000
3,9,0.00000
4,12,0.00000
...,...,...
149995,263094,0.00000
149996,263095,0.00000
149997,263096,0.00000
149998,263102,0.00000


,CLNT_ID,건강식품구매비율(금액)
0,2,0.000000
1,3,0.000000
2,4,0.000000
3,5,0.000000
4,7,0.000000
...,...,...
113099,263097,0.298718
113100,263098,0.000000
113101,263099,0.000000
113102,263100,0.000000


**[단일품목구매율]**

In [162]:
a = train.groupby(['CLNT_ID','CLAC3_NM'])['AMOUNT'].agg([('구매건수', np.size)]).reset_index()
a.query("구매건수 == 1")
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('구매건수', np.size)])
f2 = a.query("구매건수 == 1").groupby('CLNT_ID')['구매건수'].agg([('단독상품군구매수', np.sum)]).reset_index()

f = pd.merge(f,f2, on = 'CLNT_ID', how='left').fillna(0)
f['단일품목구매율'] = f['단독상품군구매수']/f['구매건수']
f = f[['CLNT_ID','단일품목구매율']]
features.append(f)


b = test.groupby(['CLNT_ID','CLAC3_NM'])['AMOUNT'].agg([('구매건수', np.size)]).reset_index()
b.query("구매건수 == 1")
f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('구매건수', np.size)])
f2_te = b.query("구매건수 == 1").groupby('CLNT_ID')['구매건수'].agg([('단독상품군구매수', np.sum)]).reset_index()

f_te = pd.merge(f_te,f2_te, on = 'CLNT_ID', how='left').fillna(0)
f_te['단일품목구매율'] = f_te['단독상품군구매수']/f_te['구매건수']
f_te = f_te[['CLNT_ID','단일품목구매율']]
features_te.append(f_te)

**[내점당구매액비율]**

In [163]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum)]).reset_index()
visits = train.groupby('CLNT_ID')['date'].agg(lambda x: x.nunique())
f2 = (train.groupby('CLNT_ID')['AMOUNT'].sum() / visits).reset_index().rename(columns={0 : "내점당구매액"})
f["내점당구매액비율"] = f2["내점당구매액"]/f["총구매액"]
f = f.drop(columns='총구매액', axis=0)
features.append(f)


f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum)]).reset_index()
visits = test.groupby('CLNT_ID')['date'].agg(lambda x: x.nunique())
f2_te = (test.groupby('CLNT_ID')['AMOUNT'].sum() / visits).reset_index().rename(columns={0 : "내점당구매액"})
f_te["내점당구매액비율"] = f2_te["내점당구매액"]/f_te["총구매액"]
f_te = f_te.drop(columns='총구매액', axis=0)
features_te.append(f_te)

**[히트 세션 평균]**

In [164]:
f = train.groupby('CLNT_ID')['HITS_SEQ'].agg([('히트세션평균',np.mean)]).reset_index()
features.append(f) ; f


f_te = test.groupby('CLNT_ID')['HITS_SEQ'].agg([('히트세션평균',np.mean)]).reset_index()
features_te.append(f_te) ; f_te

,CLNT_ID,히트세션평균
0,2,234.181818
1,3,39.500000
2,4,23.666667
3,5,53.000000
4,7,139.562500
...,...,...
113099,263097,346.230769
113100,263098,152.000000
113101,263099,275.000000
113102,263100,78.500000


**[주구매상품]**

In [165]:
f = train.groupby('CLNT_ID')['CLAC3_NM'].agg([
    ('주구매상품', lambda x: x.value_counts().index[0])
]).reset_index()
features.append(f)


f_te = test.groupby('CLNT_ID')['CLAC3_NM'].agg([
    ('주구매상품', lambda x: x.value_counts().index[0])
]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,주구매상품
0,2,유아동스포츠샌들/슬리퍼
1,3,여성일반지갑
2,4,남성티셔츠
3,5,마스카라
4,7,여성남방셔츠
...,...,...
113099,263097,여성스포츠티셔츠/탑
113100,263098,남성남방셔츠
113101,263099,남성티셔츠
113102,263100,남성티셔츠


**[식료품구매액비율]**

In [166]:
food =['과일', '음료', '냉동식품' ,'냉장식품' ,'축산물']

food_amount = train.query('CLAC1_NM == @food').groupby('CLNT_ID')['AMOUNT'].agg([
    ('식료품구매액', np.sum)
]).reset_index();food_amount
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum)]).reset_index()
f = pd.merge(f, food_amount, on = 'CLNT_ID', how = 'left').fillna(0);f

f['식료품구매액비율'] = f['식료품구매액'] / f['총구매액']

f= f.drop(columns=['총구매액', '식료품구매액'], axis=1)
features.append(f);

In [167]:
food =['과일', '음료', '냉동식품' ,'냉장식품' ,'축산물']

food_amount_te = test.query('CLAC1_NM == @food').groupby('CLNT_ID')['AMOUNT'].agg([
    ('식료품구매액', np.sum)
]).reset_index();food_amount
f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum)]).reset_index()
f_te = pd.merge(f_te, food_amount_te, on = 'CLNT_ID', how = 'left').fillna(0)

f_te['식료품구매액비율'] = f_te['식료품구매액'] / f_te['총구매액']

f_te= f_te.drop(columns=['총구매액', '식료품구매액'], axis=1)
features_te.append(f_te);f_te

,CLNT_ID,식료품구매액비율
0,2,0.0
1,3,0.0
2,4,0.0
3,5,0.0
4,7,0.0
...,...,...
113099,263097,0.0
113100,263098,0.0
113101,263099,0.0
113102,263100,0.0


**[월초-구매비율]**

In [168]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('월초-구매비율', lambda x: np.mean( x.dt.day<16)),
    ('월말-구매비율', lambda x: np.mean( x.dt.day>15))
]).reset_index()
features.append(f); features


f_te = test.groupby('CLNT_ID')['date'].agg([
    ('월초-구매비율', lambda x: np.mean( x.dt.day<16)),
    ('월말-구매비율', lambda x: np.mean( x.dt.day>15))
]).reset_index()
features_te.append(f_te); f_te

,CLNT_ID,월초-구매비율,월말-구매비율
0,2,1.000000,0.000000
1,3,1.000000,0.000000
2,4,1.000000,0.000000
3,5,1.000000,0.000000
4,7,0.937500,0.062500
...,...,...,...
113099,263097,0.692308,0.307692
113100,263098,1.000000,0.000000
113101,263099,1.000000,0.000000
113102,263100,0.000000,1.000000


**[여성선호품목구매건수]**

In [169]:
f = train.groupby('CLNT_ID')['CLAC1_NM'].agg([('여성선호품목구매건수', lambda x: list(x).count('화장품/뷰티케어')+list(x).count('유아동의류')+list(x).count('출산/육아용품')+list(x).count('퍼스널케어')+list(x).count('원예/애완')+list(x).count('여성의류'))]).reset_index()
features.append(f) ; f


f_te = test.groupby('CLNT_ID')['CLAC1_NM'].agg([('여성선호품목구매건수', lambda x: list(x).count('화장품/뷰티케어')+list(x).count('유아동의류')+list(x).count('출산/육아용품')+list(x).count('퍼스널케어')+list(x).count('원예/애완')+list(x).count('여성의류'))]).reset_index()
features_te.append(f_te) ; f_te

,CLNT_ID,여성선호품목구매건수
0,2,1
1,3,0
2,4,2
3,5,12
4,7,13
...,...,...
113099,263097,3
113100,263098,0
113101,263099,0
113102,263100,1


**[남성선호품목구매건수]**

In [170]:
f = train.groupby('CLNT_ID')['CLAC1_NM'].agg([('남성선호품목구매건수', lambda x: list(x).count('남성의류'))]).reset_index()
features.append(f) ; f


f_te = test.groupby('CLNT_ID')['CLAC1_NM'].agg([('남성선호품목구매건수', lambda x: list(x).count('남성의류'))]).reset_index()
features_te.append(f_te) ; f_te

,CLNT_ID,남성선호품목구매건수
0,2,0
1,3,0
2,4,4
3,5,0
4,7,2
...,...,...
113099,263097,0
113100,263098,6
113101,263099,112
113102,263100,1


**[상품의 종류]**

In [171]:
f = train.groupby('CLNT_ID')['PD_C'].agg([
    ('상품의종류', lambda x: x.nunique())]).reset_index()
features.append(f); f


f_te = test.groupby('CLNT_ID')['PD_C'].agg([
    ('상품의종류', lambda x: x.nunique())]).reset_index()
features_te.append(f_te); f_te

,CLNT_ID,상품의종류
0,2,3
1,3,2
2,4,2
3,5,2
4,7,7
...,...,...
113099,263097,4
113100,263098,1
113101,263099,4
113102,263100,2


**[검색키워드개수]**

In [172]:
f = train.groupby('CLNT_ID')['KWD_NM'].agg([
    ('검색키워드개수' , lambda x : x.nunique())]).reset_index()
features.append(f);display(f)


f_te = test.groupby('CLNT_ID')['KWD_NM'].agg([
    ('검색키워드개수' , lambda x : x.nunique())]).reset_index()
features_te.append(f_te);display(f_te)

,CLNT_ID,검색키워드개수
0,0,1
1,1,9
2,6,2
3,9,2
4,12,12
...,...,...
149995,263094,1
149996,263095,5
149997,263096,1
149998,263102,5


,CLNT_ID,검색키워드개수
0,2,6
1,3,4
2,4,4
3,5,4
4,7,10
...,...,...
113099,263097,7
113100,263098,6
113101,263099,7
113102,263100,2


**[주구매상품]**

In [173]:
f = train.groupby('CLNT_ID')['CLAC1_NM'].agg([('주구매상품', lambda x : x.mode()[0])]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['CLAC1_NM'].agg([('주구매상품', lambda x : x.mode()[0])]).reset_index()
features_te.append(f_te)

**[주구매월]**

In [174]:
train['month'] = train.groupby('CLNT_ID')['date'].apply(lambda x: x.dt.month)

In [175]:
test['month'] = test.groupby('CLNT_ID')['date'].apply(lambda x: x.dt.month)

**[남성취미]**

In [176]:
le = ['골프','남성스포츠화','남성케어','남성등산/아웃도어의류','캠핑','등산','남성골프의류','시계',
       '컴퓨터/노트북','컴퓨터주변기기','모바일상품권','인라인/스케이트보드/킥보드','자동차음향/가전기기','모바일기기',
       '카메라/캠코더','TV','시공/DIY가구']

In [177]:
t = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum)])
f = train.query("CLAC2_NM == @le").groupby('CLNT_ID')['AMOUNT'].agg([('남성취미구매비용', np.sum)]).reset_index()
f = pd.merge(t,f, on = 'CLNT_ID', how = 'left').fillna(0)

f = f[['CLNT_ID','남성취미구매비용']]
features.append(f)

In [178]:
t_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum)])
f_te = test.query("CLAC2_NM == @le").groupby('CLNT_ID')['AMOUNT'].agg([('남성취미구매비용', np.sum)]).reset_index()
f_te = pd.merge(t_te,f_te, on = 'CLNT_ID', how = 'left').fillna(0)

f_te = f_te[['CLNT_ID','남성취미구매비용']]
features_te.append(f_te);f_te

,CLNT_ID,남성취미구매비용
0,2,0.0
1,3,0.0
2,4,0.0
3,5,0.0
4,7,39900.0
...,...,...
113099,263097,109620.0
113100,263098,0.0
113101,263099,0.0
113102,263100,0.0


In [179]:
f = train.groupby('CLNT_ID')['month'].agg([
    ('주구매월', lambda x: x.value_counts().index[0])
]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['month'].agg([
    ('주구매월', lambda x: x.value_counts().index[0])
]).reset_index()
features_te.append(f_te);f_te

,CLNT_ID,주구매월
0,2,6
1,3,9
2,4,5
3,5,9
4,7,7
...,...,...
113099,263097,5
113100,263098,9
113101,263099,7
113102,263100,5


**[구매상품다양성비율]**

In [180]:
n = train.CLAC1_NM.nunique()
f = train.groupby('CLNT_ID')['CLAC1_NM'].agg([('구매상품다양성비율', lambda x : len(x.unique()) / n)]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['CLAC1_NM'].agg([('구매상품다양성비율', lambda x : len(x.unique()) / n)]).reset_index()
features_te.append(f_te)

**[주검색키워드]**

In [181]:
f = train.groupby('CLNT_ID')['KWD_NM'].agg([('주검색키워드', lambda x : x.mode()[0])]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['KWD_NM'].agg([('주검색키워드', lambda x : x.mode()[0])]).reset_index()
features_te.append(f_te)

**[검색키워드다양성비율]**

In [182]:
n = train.KWD_NM.nunique()

f = train.groupby('CLNT_ID')['KWD_NM'].agg([('검색키워드다양성비율', lambda x : len(x.unique()) / n)]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['KWD_NM'].agg([('검색키워드다양성비율', lambda x : len(x.unique()) / n)]).reset_index()
features_te.append(f_te)

**[검색키워드수]**

In [183]:
f = train.groupby('CLNT_ID')['KWD_NM'].agg([
    ('검색키워드수' , lambda x : x.nunique())]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID')['KWD_NM'].agg([
    ('검색키워드수' , lambda x : x.nunique())]).reset_index()
features_te.append(f_te);display(f_te)

,CLNT_ID,검색키워드수
0,0,1
1,1,9
2,6,2
3,9,2
4,12,12
...,...,...
149995,263094,1
149996,263095,5
149997,263096,1
149998,263102,5


,CLNT_ID,검색키워드수
0,2,6
1,3,4
2,4,4
3,5,4
4,7,10
...,...,...
113099,263097,7
113100,263098,6
113101,263099,7
113102,263100,2


**[여성의류구매비용]**

In [184]:
le = ['여성속옷','여성스포츠화','여성화','여성일반스포츠의류','여성의류상의',
      '여성의류전신','여성의류아우터','여성가방','여성의류하의','여성골프의류','여성지갑','여성등산/아웃도어의류',
      '패션악세서리']

In [185]:
t = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum)])
f = train.query("CLAC2_NM == @le").groupby('CLNT_ID')['AMOUNT'].agg([('여성의류구매비용', np.sum)]).reset_index()
f = pd.merge(t,f, on = 'CLNT_ID', how = 'left').fillna(0)

f = f[['CLNT_ID','여성의류구매비용']]
features.append(f);f

,CLNT_ID,여성의류구매비용
0,0,0.0
1,1,0.0
2,6,148900.0
3,9,0.0
4,12,237000.0
...,...,...
149995,263094,10000.0
149996,263095,0.0
149997,263096,47000.0
149998,263102,0.0


In [186]:
t_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum)])
f_te = test.query("CLAC2_NM == @le").groupby('CLNT_ID')['AMOUNT'].agg([('여성의류구매비용', np.sum)]).reset_index()
f_te = pd.merge(t_te,f_te, on = 'CLNT_ID', how = 'left').fillna(0)

f_te = f_te[['CLNT_ID','여성의류구매비용']]
features_te.append(f_te);f_te

,CLNT_ID,여성의류구매비용
0,2,0.0
1,3,851200.0
2,4,0.0
3,5,0.0
4,7,360000.0
...,...,...
113099,263097,103600.0
113100,263098,0.0
113101,263099,0.0
113102,263100,12900.0


**[남성의류구매비용]**

In [187]:
le=['남성일방스포츠의류','남성지갑','남성속옷','남성의류상의','남성의류하의','남성의류아우터','남성화']

In [188]:
t = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum)])
f = train.query("CLAC2_NM == @le").groupby('CLNT_ID')['AMOUNT'].agg([('남성의류구매비용', np.sum)]).reset_index()
f = pd.merge(t,f, on = 'CLNT_ID', how = 'left').fillna(0)

f = f[['CLNT_ID','남성의류구매비용']]
features.append(f);f

,CLNT_ID,남성의류구매비용
0,0,0.0
1,1,0.0
2,6,0.0
3,9,0.0
4,12,0.0
...,...,...
149995,263094,0.0
149996,263095,0.0
149997,263096,0.0
149998,263102,0.0


In [189]:
t_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum)])
f_te = test.query("CLAC2_NM == @le").groupby('CLNT_ID')['AMOUNT'].agg([('남성의류구매비용', np.sum)]).reset_index()
f_te = pd.merge(t_te,f_te, on = 'CLNT_ID', how = 'left').fillna(0)

f_te = f_te[['CLNT_ID','남성의류구매비용']]
features_te.append(f_te);f_te

,CLNT_ID,남성의류구매비용
0,2,0.0
1,3,0.0
2,4,51200.0
3,5,0.0
4,7,58000.0
...,...,...
113099,263097,0.0
113100,263098,601200.0
113101,263099,973000.0
113102,263100,9500.0


**[여성용품구매비용]**

In [190]:
le=['주방가전','수예소품','스킨케어','선케어','메이크업','조리기구','정리용품','유아스킨/바디케어','핸드/풋케어',
    '유아동침구','여성위생용품','헤어케어','수유/이유용품','유야발육용품','그릇/식기','홈웨어',
    '주방가구','바디케어','성인침구','여성양말류','미용소품','향수','이미용가전','보석']

In [191]:
t = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum)])
f = train.query("CLAC2_NM == @le").groupby('CLNT_ID')['AMOUNT'].agg([('여성용품구매비용', np.sum)]).reset_index()
f = pd.merge(t,f, on = 'CLNT_ID', how = 'left').fillna(0)

f = f[['CLNT_ID','여성용품구매비용']]
features.append(f);f

,CLNT_ID,여성용품구매비용
0,0,86500.0
1,1,728000.0
2,6,0.0
3,9,0.0
4,12,31800.0
...,...,...
149995,263094,0.0
149996,263095,155000.0
149997,263096,0.0
149998,263102,0.0


In [192]:
t_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum)])
f_te = test.query("CLAC2_NM == @le").groupby('CLNT_ID')['AMOUNT'].agg([('여성용품구매비용', np.sum)]).reset_index()
f_te = pd.merge(t_te,f_te, on = 'CLNT_ID', how = 'left').fillna(0)

f_te = f_te[['CLNT_ID','여성용품구매비용']]
features_te.append(f_te);f_te

,CLNT_ID,여성용품구매비용
0,2,20000.0
1,3,0.0
2,4,74000.0
3,5,380000.0
4,7,0.0
...,...,...
113099,263097,315000.0
113100,263098,0.0
113101,263099,0.0
113102,263100,0.0


**[최대구매품목]**

In [193]:
p = pd.pivot_table(train, index = 'CLNT_ID', columns = 'CLAC2_NM', values = 'AMOUNT', aggfunc =  np.sum)
p = p.fillna(0)
p['최대지출품목'] = p.idxmax(axis=1)
p = p['최대지출품목'].reset_index()
p
features.append(p);p

,CLNT_ID,최대지출품목
0,0,주방가전
1,1,스킨케어
2,6,여성속옷
3,9,유아동속옷
4,12,여성화
...,...,...
149995,263094,여성화
149996,263095,여아의류아우터
149997,263096,여성스포츠화
149998,263102,필기도구


In [194]:
p_te = pd.pivot_table(test, index = 'CLNT_ID', columns = 'CLAC2_NM', values = 'AMOUNT', aggfunc =  np.sum)
p_te = p_te.fillna(0)
p_te['최대지출품목'] = p_te.idxmax(axis=1)
p_te = p_te['최대지출품목'].reset_index()
p_te
features_te.append(p_te);p_te

,CLNT_ID,최대지출품목
0,2,유아동스포츠화
1,3,여성지갑
2,4,스킨케어
3,5,메이크업
4,7,여성의류상의
...,...,...
113099,263097,스킨케어
113100,263098,남성의류상의
113101,263099,남성의류상의
113102,263100,여성의류하의


**[구매당구매상품종류]**

In [195]:
a = train.groupby('CLNT_ID')['date'].agg(lambda x: x.nunique())

f = (train.groupby('CLNT_ID')['CLAC2_NM'].nunique() / a).reset_index().rename(columns={0 : '구매당구매상품종류'})
f
features.append(f)

In [196]:
a_te= test.groupby('CLNT_ID')['date'].agg(lambda x: x.nunique())

f_te = (test.groupby('CLNT_ID')['CLAC2_NM'].nunique() / a_te).reset_index().rename(columns={0 : '구매당구매상품종류'})

features_te.append(f_te);f_te

,CLNT_ID,구매당구매상품종류
0,2,1.0
1,3,1.0
2,4,1.0
3,5,2.0
4,7,1.0
...,...,...
113099,263097,2.0
113100,263098,1.0
113101,263099,2.0
113102,263100,1.0


**[구매력]**

In [197]:
f = train.groupby(['CLNT_ID','date'])['AMOUNT'].agg(np.max).reset_index().groupby('CLNT_ID')['AMOUNT'].agg([
    ('구매력지수', lambda x : np.mean(x))]).reset_index()

f = f.구매력지수.astype(int).reset_index()
f = f.rename(columns = {'index' : 'CLNT_ID'})
features.append(f)

In [198]:
f_te = test.groupby(['CLNT_ID','date'])['AMOUNT'].agg(np.max).reset_index().groupby('CLNT_ID')['AMOUNT'].agg([
    ('구매력지수', lambda x : np.mean(x))]).reset_index()

f_te = f_te.구매력지수.astype(int).reset_index()
f_te = f_te.rename(columns = {'index' : 'CLNT_ID'})
features_te.append(f_te);f_te

,CLNT_ID,구매력지수
0,0,41050
1,1,110400
2,2,24900
3,3,42000
4,4,32060
...,...,...
113099,113099,65450
113100,113100,100200
113101,113101,22000
113102,113102,11200


**[요일별구매비율]**

In [199]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('일', lambda x: np.mean( x.dt.day == 0 )),
    ('월', lambda x: np.mean( x.dt.day == 1 )),
    ('화', lambda x: np.mean( x.dt.day == 2 )),
    ('수', lambda x: np.mean( x.dt.day == 3 )),
    ('목', lambda x: np.mean( x.dt.day == 4 )),
    ('금', lambda x: np.mean( x.dt.day == 5 )),
    ('토', lambda x: np.mean( x.dt.day == 6 ))
]).reset_index();f
features.append(f)

In [200]:
f_te = test.groupby('CLNT_ID')['date'].agg([
    ('일', lambda x: np.mean( x.dt.day == 0 )),
    ('월', lambda x: np.mean( x.dt.day == 1 )),
    ('화', lambda x: np.mean( x.dt.day == 2 )),
    ('수', lambda x: np.mean( x.dt.day == 3 )),
    ('목', lambda x: np.mean( x.dt.day == 4 )),
    ('금', lambda x: np.mean( x.dt.day == 5 )),
    ('토', lambda x: np.mean( x.dt.day == 6 ))
]).reset_index();f_te
features_te.append(f_te)

**[브랜드다양성]**

In [201]:
n = train.PD_BRA_NM.nunique()

f = train.groupby('CLNT_ID')['PD_BRA_NM'].agg([('브랜드다양성', lambda x: len(x.unique()) / n)]).reset_index()
features.append(f) ; f

n = test.PD_BRA_NM.nunique()
f_te = test.groupby('CLNT_ID')['PD_BRA_NM'].agg([('브랜드다양성', lambda x: len(x.unique()) / n)]).reset_index()
features_te.append(f_te) ; f_te

,CLNT_ID,브랜드다양성
0,2,0.000347
1,3,0.000231
2,4,0.000231
3,5,0.000231
4,7,0.000810
...,...,...
113099,263097,0.000463
113100,263098,0.000116
113101,263099,0.000116
113102,263100,0.000231


**[구매세션 평균]**

In [202]:
f=train.groupby('CLNT_ID')['SESS_SEQ'].agg([('구매세션평균',np.mean)]).reset_index()
features.append(f) ; f

f_te=test.groupby('CLNT_ID')['SESS_SEQ'].agg([('구매세션평균',np.mean)]).reset_index()
features_te.append(f_te) ; f_te

,CLNT_ID,구매세션평균
0,2,37.272727
1,3,1.000000
2,4,73.666667
3,5,11.000000
4,7,99.562500
...,...,...
113099,263097,362.230769
113100,263098,342.000000
113101,263099,5.000000
113102,263100,133.500000


**[최대검색건수]**

In [203]:
f = train.groupby('CLNT_ID')['SESS_SEQ'].agg([('최대검색건수', np.max)]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['SESS_SEQ'].agg([('최대검색건수', np.max)]).reset_index()
features_te.append(f_te)

**[최소검색건수]**

In [204]:
f = train.groupby('CLNT_ID')['SESS_SEQ'].agg([('최대검색건수', np.min)]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['SESS_SEQ'].agg([('최대검색건수', np.min)]).reset_index()
features_te.append(f_te)

**[상품금액평균]**

In [205]:
f = train.groupby('CLNT_ID')['PD_BUY_AM'].agg([('상품금액평균',lambda x : np.mean(x))]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['PD_BUY_AM'].agg([('상품금액평균',lambda x : np.mean(x))]).reset_index()
features_te.append(f_te)

**[월초구매비율]**

In [206]:
f = train.groupby('CLNT_ID')['date'].agg([('월초-구매비율', lambda x : np.mean(x.dt.day<16))]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['date'].agg([('월초-구매비율', lambda x : np.mean(x.dt.day<16))]).reset_index()
features_te.append(f_te)

**[월말구매비율]**

In [207]:
f = train.groupby('CLNT_ID')['date'].agg([('월초-구매비율', lambda x : np.mean(x.dt.day>15))]).reset_index()
features.append(f)

f_te = test.groupby('CLNT_ID')['date'].agg([('월초-구매비율', lambda x : np.mean(x.dt.day>15))]).reset_index()
features_te.append(f_te)

### Categorical Variable

**[주구매요일]**

**[주구매지역]**

In [208]:
f = train.groupby('CLNT_ID')['CITY_NM'].agg([('주구매지역', lambda x: x.value_counts().index[0])]).reset_index()
features.append(f) ; display(f)


f_te = test.groupby('CLNT_ID')['CITY_NM'].agg([('주구매지역', lambda x: x.value_counts().index[0])]).reset_index()
features_te.append(f_te) ; display(f_te)

,CLNT_ID,주구매지역
0,0,Bucheon-si
1,1,Seoul
2,6,Wanju-gun
3,9,Gwangju
4,12,Namyangju-si
...,...,...
149995,263094,Seoul
149996,263095,Busan
149997,263096,Gunsan-si
149998,263102,Seoul


,CLNT_ID,주구매지역
0,2,Ulsan
1,3,Incheon
2,4,Seoul
3,5,Seoul
4,7,Daejeon
...,...,...
113099,263097,Seongnam-si
113100,263098,Anyang
113101,263099,Gimcheon-si
113102,263100,Seoul


**[주사용기기]**

In [211]:
f = train.groupby('CLNT_ID')['DVC_CTG_NM'].agg([('주 사용 기기', lambda x: x.value_counts().index[0])]).reset_index()
features.append(f) ; f

f_te = test.groupby('CLNT_ID')['DVC_CTG_NM'].agg([('주 사용 기기', lambda x: x.value_counts().index[0])]).reset_index()
features_te.append(f_te) ; f_te

,CLNT_ID,주 사용 기기
0,2,mobile
1,3,mobile
2,4,mobile
3,5,mobile
4,7,mobile
...,...,...
113099,263097,mobile
113100,263098,mobile
113101,263099,mobile
113102,263100,mobile


### Merge

In [212]:
data = pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()})

for f in features :
    data = pd.merge(data, f, how='left')
data = data.fillna(0)
data.to_csv('data.csv', index=False, encoding='cp949')

In [213]:
data_te = pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()})

for f_te in features_te :
    data_te = pd.merge(data_te, f_te, how='left')
    
data_te.to_csv('data_te.csv', index=False, encoding='cp949')